In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
house_df = pd.read_csv("train.csv")

In [3]:
house_df.shape

(1460, 81)

In [4]:
house_df.head()
#pd.set_option("max_columns",None)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
house_df.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [6]:
#EDA
house_df.drop(['Id'],axis=1,inplace=True)

In [7]:
house_df.shape

(1460, 80)

In [8]:
#Checking datatype counts
house_df.dtypes.value_counts()

object     43
int64      34
float64     3
dtype: int64

In [9]:
target = pd.DataFrame(house_df['SalePrice'])
target.dtypes

SalePrice    int64
dtype: object

In [10]:
independent_df = house_df.drop('SalePrice',axis=1)

In [11]:
independent_df.dtypes.value_counts()

object     43
int64      33
float64     3
dtype: int64

In [12]:
independent_df.isnull().sum().sort_values(ascending=False,kind='mergesort')

PoolQC         1453
MiscFeature    1406
Alley          1369
Fence          1179
FireplaceQu     690
               ... 
LotShape          0
Street            0
LotArea           0
MSZoning          0
MSSubClass        0
Length: 79, dtype: int64

In [13]:
#Finding columns which have more than 60% missing values
empty_df = independent_df.dropna(axis=1,thresh=(len(independent_df) * 0.6))
print(empty_df.shape)
print(independent_df.columns.difference(empty_df.columns))

#To save memory removing irrelavent variables
empty_df = None

(1460, 74)
Index(['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC'], dtype='object')


In [14]:
#EDA
#Removing columns which have more than 60% missing values
independent_df.drop(labels=['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC'],axis=1,inplace=True)
independent_df.shape

(1460, 74)

In [15]:
independent_df.isnull().sum().sort_values(ascending=False)

LotFrontage     259
GarageFinish     81
GarageType       81
GarageCond       81
GarageQual       81
               ... 
CentralAir        0
HeatingQC         0
Heating           0
SaleType          0
MSSubClass        0
Length: 74, dtype: int64

In [16]:
#EDA
#Removing 85% low variance constant columns
low_var_col = []
for colname,colvalues in independent_df.items():
    column_count = colvalues.value_counts()[abs(colvalues.value_counts(normalize=True)) > 0.85].count()
    if column_count == 1:
        low_var_col.append(colname)
        independent_df.drop(colname, axis=1,inplace=True)
        
print(low_var_col)

['Street', 'LandContour', 'Utilities', 'LandSlope', 'Condition1', 'Condition2', 'RoofMatl', 'ExterCond', 'BsmtCond', 'BsmtFinType2', 'BsmtFinSF2', 'Heating', 'CentralAir', 'Electrical', 'LowQualFinSF', 'BsmtHalfBath', 'KitchenAbvGr', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SaleType']


In [17]:
independent_df.shape

(1460, 47)

In [18]:
#EDA
numerical_df = pd.DataFrame(independent_df.select_dtypes(exclude='object'))
categorical_df = pd.DataFrame(independent_df.select_dtypes(include='object'))
print("Numerical Data Types " + str(numerical_df.dtypes.value_counts().sum()))
print("Categorical Datatypes " + str(categorical_df.dtypes.value_counts().sum()))

Numerical Data Types 27
Categorical Datatypes 20


In [19]:
numerical_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,...,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706,150,...,3,8,0,2003.0,2,548,0,61,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978,284,...,3,6,1,1976.0,2,460,298,0,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486,434,...,3,6,1,2001.0,2,608,0,42,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216,540,...,3,7,1,1998.0,3,642,0,35,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655,490,...,4,9,1,2000.0,3,836,192,84,12,2008


In [20]:
numerical_df.isnull().sum().sort_values(ascending=False)

LotFrontage     259
GarageYrBlt      81
MasVnrArea        8
YrSold            0
2ndFlrSF          0
LotArea           0
OverallQual       0
OverallCond       0
YearBuilt         0
YearRemodAdd      0
BsmtFinSF1        0
BsmtUnfSF         0
TotalBsmtSF       0
1stFlrSF          0
GrLivArea         0
MoSold            0
BsmtFullBath      0
FullBath          0
HalfBath          0
BedroomAbvGr      0
TotRmsAbvGrd      0
Fireplaces        0
GarageCars        0
GarageArea        0
WoodDeckSF        0
OpenPorchSF       0
MSSubClass        0
dtype: int64

In [29]:
numerical_df['LotFrontage'].describe()

count    1201.000000
mean       70.049958
std        24.284752
min        21.000000
25%        59.000000
50%        69.000000
75%        80.000000
max       313.000000
Name: LotFrontage, dtype: float64

In [34]:
numerical_df['MasVnrArea'].describe()

count    1452.000000
mean      103.685262
std       181.066207
min         0.000000
25%         0.000000
50%         0.000000
75%       166.000000
max      1600.000000
Name: MasVnrArea, dtype: float64

In [21]:
#EDA
#Filling empty values with median
impute_value_LotFrontage=numerical_df['LotFrontage'].median()
impute_value_MasVnrArea=numerical_df['MasVnrArea'].median()
numerical_df['LotFrontage'].fillna(impute_value_LotFrontage,inplace=True)
numerical_df['MasVnrArea'].fillna(impute_value_MasVnrArea,inplace=True)

In [22]:
numerical_df.isnull().sum().sort_values(ascending=False)

GarageYrBlt     81
YrSold           0
2ndFlrSF         0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
GrLivArea        0
MoSold           0
BsmtFullBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
MSSubClass       0
dtype: int64

In [53]:
numerical_df.columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'MoSold', 'YrSold'],
      dtype='object')

In [23]:
import datetime
now = datetime.datetime.now()

In [24]:
#EDA
#Converting year values to meaningful numbers
numerical_df['YearBuilt'].update(now.year - numerical_df['YearBuilt'])
numerical_df['YearRemodAdd'].update(now.year - numerical_df['YearRemodAdd'])
numerical_df['GarageYrBlt'].update(now.year - numerical_df['GarageYrBlt'])
numerical_df['YrSold'].update(now.year - numerical_df['YrSold'])

In [71]:
print(numerical_df['GarageYrBlt'].sort_values())

378     10.0
819     10.0
987     10.0
412     11.0
157     11.0
        ... 
1349     NaN
1407     NaN
1449     NaN
1450     NaN
1453     NaN
Name: GarageYrBlt, Length: 1460, dtype: float64


In [25]:
#EDA
#Filling empty values with median
impute_value_GarageYrBlt=numerical_df['GarageYrBlt'].median()
numerical_df['GarageYrBlt'].fillna(impute_value_GarageYrBlt,inplace=True)

In [73]:
numerical_df.isnull().sum().sort_values(ascending=False)

YrSold          0
2ndFlrSF        0
LotFrontage     0
LotArea         0
OverallQual     0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
MasVnrArea      0
BsmtFinSF1      0
BsmtUnfSF       0
TotalBsmtSF     0
1stFlrSF        0
GrLivArea       0
MoSold          0
BsmtFullBath    0
FullBath        0
HalfBath        0
BedroomAbvGr    0
TotRmsAbvGrd    0
Fireplaces      0
GarageYrBlt     0
GarageCars      0
GarageArea      0
WoodDeckSF      0
OpenPorchSF     0
MSSubClass      0
dtype: int64

In [26]:
#Collenearity check with target variable
coll_df = pd.concat([numerical_df,target],axis=1)

In [90]:
coll_df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,...,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,17,17,196.0,706,150,...,8,0,17.0,2,548,0,61,2,12,208500
1,20,80.0,9600,6,8,44,44,0.0,978,284,...,6,1,44.0,2,460,298,0,5,13,181500
2,60,68.0,11250,7,5,19,18,162.0,486,434,...,6,1,19.0,2,608,0,42,9,12,223500
3,70,60.0,9550,7,5,105,50,0.0,216,540,...,7,1,22.0,3,642,0,35,2,14,140000
4,60,84.0,14260,8,5,20,20,350.0,655,490,...,9,1,20.0,3,836,192,84,12,12,250000


In [27]:
corr_matrix = coll_df.corr()

In [28]:
for i in range(corr_matrix.shape[0]):
    for j in range(i+1, corr_matrix.shape[0]):
        if abs(corr_matrix.iloc[i,j]) >= 0.7:
               print(corr_matrix.index[i]+','+corr_matrix.index[j]+ '-->' +str(corr_matrix.iloc[i,j]))

OverallQual,SalePrice-->0.7909816005838047
YearBuilt,GarageYrBlt-->0.7771817647425328
TotalBsmtSF,1stFlrSF-->0.8195299750050355
GrLivArea,TotRmsAbvGrd-->0.8254893743088377
GrLivArea,SalePrice-->0.7086244776126511
GarageCars,GarageArea-->0.8824754142814603


In [29]:
#EDA
#Picking columns based on Correlation and multicollinearity values above from numerical dataframe
numerical_df = numerical_df[['OverallQual','YearBuilt','TotalBsmtSF','GrLivArea','GarageArea']]
numerical_df.shape

(1460, 5)

In [116]:
numerical_df.head()

,OverallQual,YearBuilt,TotalBsmtSF,GrLivArea,GarageArea
0,7,17,856,1710,548
1,6,44,1262,1262,460
2,7,19,920,1786,608
3,7,105,756,1717,642
4,8,20,1145,2198,836


In [254]:
categorical_df.isnull().sum().sort_values(ascending=False)

GarageType       81
GarageFinish     81
BsmtExposure     38
BsmtFinType1     37
BsmtQual         37
MasVnrType        8
RoofStyle         0
LotShape          0
LotConfig         0
Neighborhood      0
BldgType          0
HouseStyle        0
SaleCondition     0
Exterior1st       0
Exterior2nd       0
ExterQual         0
Foundation        0
HeatingQC         0
KitchenQual       0
MSZoning          0
dtype: int64

In [30]:
print("GarageType --> "+str(categorical_df['GarageType'].unique()))
print("GarageFinish --> "+str(categorical_df['GarageFinish'].unique()))
print("BsmtExposure --> "+str(categorical_df['BsmtExposure'].unique()))
print("BsmtFinType1 --> "+str(categorical_df['BsmtFinType1'].unique()))
print("BsmtQual --> "+str(categorical_df['BsmtQual'].unique()))
print("MasVnrType --> "+str(categorical_df['MasVnrType'].unique()))

GarageType --> ['Attchd' 'Detchd' 'BuiltIn' 'CarPort' nan 'Basment' '2Types']
GarageFinish --> ['RFn' 'Unf' 'Fin' nan]
BsmtExposure --> ['No' 'Gd' 'Mn' 'Av' nan]
BsmtFinType1 --> ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' nan 'LwQ']
BsmtQual --> ['Gd' 'TA' 'Ex' nan 'Fa']
MasVnrType --> ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]


In [31]:
#EDA
#NG for No Garage
#NB for No basement for all missing values
#Nothing
#Filling empty values with median
impute_value_Garage="NG"
impute_value_Basement="NB"
impute_value_MasVnrType="None"
categorical_df['GarageType'].fillna(impute_value_Garage,inplace=True)
categorical_df['GarageFinish'].fillna(impute_value_Garage,inplace=True)
categorical_df['BsmtExposure'].fillna(impute_value_Basement,inplace=True)
categorical_df['BsmtFinType1'].fillna(impute_value_Basement,inplace=True)
categorical_df['BsmtQual'].fillna(impute_value_Basement,inplace=True)
categorical_df['MasVnrType'].fillna(impute_value_MasVnrType,inplace=True)

In [259]:
categorical_df.isnull().sum().sort_values(ascending=False)

SaleCondition    0
GarageFinish     0
LotShape         0
LotConfig        0
Neighborhood     0
BldgType         0
HouseStyle       0
RoofStyle        0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
Foundation       0
BsmtQual         0
BsmtExposure     0
BsmtFinType1     0
HeatingQC        0
KitchenQual      0
GarageType       0
MSZoning         0
dtype: int64

In [260]:
categorical_df.shape

(1460, 20)

In [32]:
#Performing Annova test to find out and pick columns that have variance against target variable
import statsmodels.stats.multicomp as multi

In [261]:
mc = multi.MultiComparison(target['SalePrice'], categorical_df['SaleCondition'])
result = mc.tukeyhsd()
result.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
Abnorml,AdjLand,-42401.6238,0.8596,-150038.5548,65235.3073,False
Abnorml,Alloca,20850.7929,0.9,-43617.3636,85318.9494,False
Abnorml,Family,3073.3762,0.9,-48600.8926,54747.645,False
Abnorml,Normal,28675.5958,0.0026,6799.3487,50551.8428,True
Abnorml,Partial,125765.1282,0.001,97516.5974,154013.6591,True
AdjLand,Alloca,63252.4167,0.6539,-58645.6204,185150.4537,False
AdjLand,Family,45475.0,0.8607,-70167.6318,161117.6318,False
AdjLand,Normal,71077.2195,0.3929,-34665.6688,176820.1078,False
AdjLand,Partial,168166.752,0.001,60924.1871,275409.3169,True
Alloca,Family,-17777.4167,0.9,-94872.5045,59317.6712,False


In [262]:
len(result.reject[result.reject==True])/len(result.reject)

0.4

In [33]:
select_annova_columns_for_onehot = set()
select_annova_columns_for_label = set()
select_annova_columns_for_dropping = set()

for column in categorical_df:
    mc = multi.MultiComparison(target['SalePrice'], categorical_df[column])
    result = mc.tukeyhsd()
    
    reject_col = abs(len(result.reject[result.reject==True])/len(result.reject))
    
    if(reject_col == 1 or reject_col == 0.5):
        select_annova_columns_for_label.add(column)
    elif(reject_col > 0.5):
        select_annova_columns_for_onehot.add(column)
    else:
        select_annova_columns_for_dropping.add(column)
        
print("One hot encoding columns --> "+str(select_annova_columns_for_onehot))
print("Label encoding columns --> "+str(select_annova_columns_for_label))
print("Columns to be dropped --> "+str(select_annova_columns_for_dropping))

One hot encoding columns --> {'BldgType', 'Neighborhood', 'BsmtExposure', 'MasVnrType', 'MSZoning', 'BsmtQual', 'BsmtFinType1', 'GarageType'}
Label encoding columns --> {'LotShape', 'ExterQual', 'GarageFinish', 'KitchenQual'}
Columns to be dropped --> {'HouseStyle', 'Exterior2nd', 'SaleCondition', 'Foundation', 'HeatingQC', 'LotConfig', 'RoofStyle', 'Exterior1st'}


In [34]:
#EDA
#Dropping low relation columns
categorical_df.drop(labels=['Exterior2nd', 'Foundation', 'HouseStyle', 'RoofStyle', 'HeatingQC', \
                            'SaleCondition', 'Exterior1st', 'LotConfig'],axis=1,inplace=True)

In [35]:
categorical_df.shape

(1460, 12)

In [266]:
for column in categorical_df:
    col_details = categorical_df[column].unique()
    print(column +" --> " + str(len(col_details)) + "with unique values --> "+str(col_details))

MSZoning --> 5with unique values --> ['RL' 'RM' 'C (all)' 'FV' 'RH']
LotShape --> 4with unique values --> ['Reg' 'IR1' 'IR2' 'IR3']
Neighborhood --> 25with unique values --> ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
BldgType --> 5with unique values --> ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
MasVnrType --> 4with unique values --> ['BrkFace' 'None' 'Stone' 'BrkCmn']
ExterQual --> 4with unique values --> ['Gd' 'TA' 'Ex' 'Fa']
BsmtQual --> 5with unique values --> ['Gd' 'TA' 'Ex' 'NB' 'Fa']
BsmtExposure --> 5with unique values --> ['No' 'Gd' 'Mn' 'Av' 'NB']
BsmtFinType1 --> 7with unique values --> ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' 'NB' 'LwQ']
KitchenQual --> 4with unique values --> ['Gd' 'TA' 'Ex' 'Fa']
GarageType --> 7with unique values --> ['Attchd' 'Detchd' 'BuiltIn' 'CarPort' 'NG' 'Ba

In [36]:
#EDA
#Label Encoding

cat_list_dict1 = {'GarageFinish': ['RFn','Unf', 'Fin', 'NG'],
                'LotShape': ['Reg', 'IR1', 'IR2', 'IR3'],
                'KitchenQual': ['Gd', 'TA', 'Ex', 'Fa'],
                'ExterQual': ['Gd', 'TA', 'Ex', 'Fa'],
                'BsmtQual': ['Gd', 'TA', 'Ex', 'NB', 'Fa']}

In [37]:
def convertCategorisToCodes(columns, df):
    for column in columns:
        df[column] = pd.Categorical(df[column], categories = cat_list_dict1[column])
        df[column] = df[column].cat.codes

In [38]:
#Label encoding columns --> {'GarageFinish', 'LotShape', 'KitchenQual', 'ExterQual'}
#Adding additional column 'BsmtQual' as well due to order present
convertCategorisToCodes(['GarageFinish'],categorical_df)
convertCategorisToCodes(['LotShape'],categorical_df)
convertCategorisToCodes(['KitchenQual'],categorical_df)
convertCategorisToCodes(['ExterQual'],categorical_df)
convertCategorisToCodes(['BsmtQual'],categorical_df)

In [269]:
categorical_df.head()

,MSZoning,LotShape,Neighborhood,BldgType,MasVnrType,ExterQual,BsmtQual,BsmtExposure,BsmtFinType1,KitchenQual,GarageType,GarageFinish
0,RL,0,CollgCr,1Fam,BrkFace,0,0,No,GLQ,0,Attchd,0
1,RL,0,Veenker,1Fam,None,1,0,Gd,ALQ,1,Attchd,0
2,RL,1,CollgCr,1Fam,BrkFace,0,0,Mn,GLQ,0,Attchd,0
3,RL,1,Crawfor,1Fam,None,1,1,No,ALQ,0,Detchd,1
4,RL,1,NoRidge,1Fam,BrkFace,0,0,Av,GLQ,0,Attchd,0


In [39]:
#EDA
#One hot encoding

cat_list_dict2 = {'BldgType': ['1Fam','2fmCon','Duplex','TwnhsE','Twnhs'], 
                  'BsmtFinType1': ['GLQ','ALQ','Unf','Rec','BLQ','NB','LwQ'], 
                  'GarageType': ['Attchd','Detchd','BuiltIn','CarPort','NG','Basment','2Types'], 
                  'BsmtExposure': ['No','Gd','Mn','Av','NB'], 
                  'MSZoning': ['RL','RM','C (all)','FV','RH'], 
                  'MasVnrType': ['BrkFace','None','Stone','BrkCmn'], 
                  'Neighborhood': ['CollgCr','Veenker','Crawfor','NoRidge','Mitchel','Somerst','NWAmes'
 'OldTown','BrkSide','Sawyer','NridgHt','NAmes','SawyerW','IDOTRR'
 'MeadowV','Edwards','Timber','Gilbert','StoneBr','ClearCr','NPkVill'
 'Blmngtn','BrDale','SWISU','Blueste']}

In [40]:
#One hot encoding columns --> {'BldgType', 'BsmtFinType1', 'GarageType', 'BsmtExposure', 'MSZoning', 'MasVnrType', 'Neighborhood'}

for key_name in cat_list_dict2.keys():
    categorical_df[key_name] = pd.Categorical(categorical_df[key_name], categories = cat_list_dict2[key_name])
    categorical_df = pd.get_dummies(categorical_df, columns=[key_name], prefix=[key_name], drop_first=True)

In [300]:
categorical_df.head()

,LotShape,ExterQual,BsmtQual,KitchenQual,GarageFinish,BldgType_2fmCon,BldgType_Duplex,BldgType_TwnhsE,BldgType_Twnhs,BsmtFinType1_ALQ,...,Neighborhood_IDOTRRMeadowV,Neighborhood_Edwards,Neighborhood_Timber,Neighborhood_Gilbert,Neighborhood_StoneBr,Neighborhood_ClearCr,Neighborhood_NPkVillBlmngtn,Neighborhood_BrDale,Neighborhood_SWISU,Neighborhood_Blueste
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
#Model fit with Train data check
from sklearn.linear_model import LinearRegression

In [42]:
train_X = pd.concat([numerical_df,categorical_df], axis=1)
train_y = target

In [43]:
lr_model = LinearRegression()
lr_model.fit(train_X, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [44]:
train_prediction = lr_model.predict(train_X)

In [45]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [46]:
train_r2_score = r2_score(train_y, train_prediction)
train_mse = mean_squared_error(train_y, train_prediction)

print("Train R2 Score : ", train_r2_score)
print("Train Root Mean Squared Error : ", np.sqrt(train_mse))

Train R2 Score :  0.8475567167080034
Train Root Mean Squared Error :  31006.895345699053


# Testing using Test Dataset

In [47]:
test_df = pd.read_csv("test.csv")

In [323]:
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [48]:
test_df.shape

(1459, 80)

## Performing EDA as in train for all features in test dataset

In [49]:
test_df.drop(['Id'],axis=1,inplace=True)
test_df.drop(labels=['Alley', 'Fence', 'FireplaceQu', 'MiscFeature', 'PoolQC'],axis=1,inplace=True)
test_df.drop(labels=['Street', 'LandContour', 'Utilities', 'LandSlope', 'Condition1', 'Condition2', 'RoofMatl', 
                     'ExterCond', 'BsmtCond', 'BsmtFinType2', 'BsmtFinSF2', 'Heating', 'CentralAir', 'Electrical', 
                     'LowQualFinSF', 'BsmtHalfBath', 'KitchenAbvGr', 'Functional', 'GarageQual', 'GarageCond', 
                     'PavedDrive', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 
                     'SaleType'], axis=1,inplace=True)

In [50]:
#EDA
numerical_df = pd.DataFrame(test_df.select_dtypes(exclude='object'))
categorical_df = pd.DataFrame(test_df.select_dtypes(include='object'))

In [51]:
numerical_df['LotFrontage'].fillna(impute_value_LotFrontage,inplace=True)
numerical_df['MasVnrArea'].fillna(impute_value_MasVnrArea,inplace=True)

numerical_df['YearBuilt'].update(now.year - numerical_df['YearBuilt'])
numerical_df['YearRemodAdd'].update(now.year - numerical_df['YearRemodAdd'])
numerical_df['GarageYrBlt'].update(now.year - numerical_df['GarageYrBlt'])
numerical_df['YrSold'].update(now.year - numerical_df['YrSold'])

numerical_df['GarageYrBlt'].fillna(impute_value_GarageYrBlt,inplace=True)

In [52]:
#Picking columns based on Correlation and multicollinearity values above from numerical dataframe
numerical_df = numerical_df[['OverallQual','YearBuilt','TotalBsmtSF','GrLivArea','GarageArea']]

In [53]:
categorical_df['GarageType'].fillna(impute_value_Garage,inplace=True)
categorical_df['GarageFinish'].fillna(impute_value_Garage,inplace=True)
categorical_df['BsmtExposure'].fillna(impute_value_Basement,inplace=True)
categorical_df['BsmtFinType1'].fillna(impute_value_Basement,inplace=True)
categorical_df['BsmtQual'].fillna(impute_value_Basement,inplace=True)
categorical_df['MasVnrType'].fillna(impute_value_MasVnrType,inplace=True)

categorical_df.drop(labels=['Exterior2nd', 'Foundation', 'HouseStyle', 'RoofStyle', 'HeatingQC', \
                            'SaleCondition', 'Exterior1st', 'LotConfig'],axis=1,inplace=True)

In [54]:
convertCategorisToCodes(['GarageFinish'],categorical_df)
convertCategorisToCodes(['LotShape'],categorical_df)
convertCategorisToCodes(['KitchenQual'],categorical_df)
convertCategorisToCodes(['ExterQual'],categorical_df)
convertCategorisToCodes(['BsmtQual'],categorical_df)

In [55]:
for key_name in cat_list_dict2.keys():
    categorical_df[key_name] = pd.Categorical(categorical_df[key_name], categories = cat_list_dict2[key_name])
    categorical_df = pd.get_dummies(categorical_df, columns=[key_name], prefix=[key_name], drop_first=True)

In [56]:
test_X = pd.concat([numerical_df,categorical_df], axis=1)

In [59]:
test_prediction = lr_model.predict(test_X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').